In [ ]:
filename = "day13.example.input"
filename = "day13.input"

with open(filename) as file:
    timestamp = int(file.readline())
    bus_ids = [int(bus_id) for bus_id in file.readline().split(",") if bus_id != "x"]

# Part 1

In [ ]:
next_departure = {bus_id: timestamp + bus_id - timestamp % bus_id for bus_id in bus_ids}
first_bus_id = min(next_departure, key=next_departure.get)
waiting_time = next_departure[first_bus_id] - timestamp

first_bus_id*waiting_time

# Part 2

In [ ]:
with open(filename) as file:
    timestamp = int(file.readline())        
    bus_id_order = {
        int(bus_id): bus_order
        for bus_order, bus_id in enumerate(file.readline().split(","))
        if bus_id != "x"
    }

bus_id_order

## Idea for solving:

We want to find a timestamp when the waiting time for `bus_id` is `bus_id_order[bus_id]`, and we want this to be true for all busses at the same time. The waiting time until a given `bus_id` arrives is:
```
waiting_time = bus_id - (timestamp % bus_id or bus_id)
```

* The trick with ...`or bus_id` is needed because: At departure time, `timestamp % bus_id == 0`, and subtracting 0 leaves us with a `waiting_time` of `bus_id`. In those cases, we instead want to subtract `bus_id` which gives a `waiting_time` of 0.

* In the full input, the required waiting time is sometimes higher than the `bus_id`, e.g. bus 13 has a required waiting time of 47. But since bus 13 leaves every 13 minutes, this is the same as requiring `47 % 13 = 8` minutes waiting time.

We solve the problem one bus at the time:

1. Increase timestamp with 1 until the required waiting time is fulfilled for the first bus.
1. When searching for a valid timestamp for the next bus, we need to move in step size of `bus_id` of the first bus. That ensures that the first requirement will still be fulfilled.
1. After having found a valid timestamp for the two first buses, we now move with a step size of `bus_id_0 * bus_id_1`. That ensures that the two first requirements are still fulfilled.
1. And so on...

To speed things up even more, we start with the largest `bus_id`, to ensure that we move with the largest step size possible.

In [ ]:
sorted_bus_ids = sorted(bus_id_order.keys(), reverse=True)

step_size = 1
for bus_id in sorted_bus_ids:
    target = bus_id_order[bus_id] % bus_id
    while bus_id - (timestamp % bus_id or bus_id) != target:
        timestamp += step_size
    step_size *= bus_id
    print(f"Solved for bus {bus_id} at timestamp {timestamp:,}. Next step size is {step_size:,}")

timestamp